# Урок 5. Задача оттока: варианты постановки, возможные способы решения
1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности  
***
Ссылки  
http://hyperopt.github.io/hyperopt/
https://arxiv.org/pdf/1907.03947.pdf
https://arxiv.org/pdf/1802.02301.pdf
https://arxiv.org/list/stat.ML/recent
https://scikit-learn.org/stable/modules/grid_search.html
https://scikit-learn.org/stable/modules/compose.html